**Interpretation of All Models (With Discussion of R² and Within R²)**

1. **Model 1: No Fixed Effects**  
   - **Specification**: `fatal_rate ~ beertax` (no FEs)  
   - **Coefficient on beertax**: \(\mathbf{+3.65}\)  
   - **R²: 0.093**  
   - *Interpretation*: In a naive OLS, higher beer taxes correlate with *higher* fatality rates. This likely reflects cross-state differences (e.g., some states have high taxes and other unobserved traits that push fatality rates up or down).  
   - *Why R² is relatively low*: We are only using one regressor (beer tax) to explain all variation in fatality rates across states and time.

2. **Model 2: State Fixed Effects**  
   - **Specification**: `fatal_rate ~ beertax | state`  
   - **Coefficient on beertax**: \(\mathbf{-6.56}\)  
   - **R²: 0.905, Within R²: 0.041**  
   - *Interpretation*: Once we partial out each state’s *average* fatality rate (the state FE), the coefficient flips sign and becomes *negative*. This suggests that *within* a given state over time, an increase in beer tax is associated with *lower* fatality rates.  
   - *Why R² is so high but Within R² is small*:  
     - The **overall R² (0.905)** includes large cross-state mean differences captured by the fixed effects, so the model “explains” most total variation once we let each state have its own intercept.  
     - The **Within R² (0.041)** indicates how much *within-state* (year-to-year) variation in fatality rates is explained by changes in beer tax. Because many factors drive fatalities, only ~4.1% of that within-state variation is accounted for by beer tax alone.

3. **Model 3: Manual State Dummies**  
   - **Specification**: `fatal_rate ~ beertax + C(state)`  
   - **Coefficient on beertax**: \(\mathbf{-6.56}\)  
   - *Interpretation*: This matches Model 2 exactly, proving that adding a full set of dummies for `state` is algebraically the same as specifying `| state` in PyFixest.

4. **Model 4: Year Fixed Effects Only**  
   - **Specification**: `fatal_rate ~ beertax | year`  
   - **Coefficient on beertax**: \(\mathbf{+3.66}\)  
   - **R²: 0.099, Within R²: 0.095**  
   - *Interpretation*: Controlling only for year-to-year national shocks does not eliminate cross-state differences, so the coefficient remains positive and fairly similar to the naive model. This means that simply removing “time effects” is insufficient—state-level differences are still unaccounted for.

5. **Model 5: Manual Year Dummies**  
   - **Specification**: `fatal_rate ~ beertax + C(year)`  
   - **Coefficient on beertax**: \(\mathbf{+3.66}\)  
   - *Interpretation*: Identical to Model 4, confirming that `C(year)` is the manual version of `| year`.

6. **Model 6: State + Year Fixed Effects**  
   - **Specification**: `fatal_rate ~ beertax | state + year`  
   - **Coefficient on beertax**: \(\mathbf{-6.40}\)  
   - **R²: 0.909, Within R²: 0.036**  
   - *Interpretation*: Now we partial out both *state* and *year* effects. We again find a negative coefficient, close to -6.4. This indicates that, net of state-specific levels and year-specific shocks, higher beer taxes correlate with lower fatality rates.  
   - *Why the Within R² is small*: Just 3.6% of the within-state-year variation is explained by beer tax. The overall R² is large because each state and year’s mean levels are removed, so the total model can look extremely explanatory once you soak up those big cross-sectional and time-level differences.

7. **Model 7: Manual Dummies for State + Year**  
   - **Specification**: `fatal_rate ~ beertax + C(state) + C(year)`  
   - **Coefficient on beertax**: \(\mathbf{-6.40}\)  
   - *Interpretation*: Same result as Model 6, confirming that explicit dummies for both `state` and `year` yield identical estimates to the “short-form” fixed effects notation.

---

### Key Takeaways

- **Sign Flip**: Moving from no fixed effects to including *state* fixed effects changes the beer-tax coefficient from positive to negative, revealing how omitted state-level variables confound the naive OLS.  
- **Year FE Alone**: Merely adding year indicators does *not* remove the cross-state heterogeneity, so the sign remains positive.  
- **State + Year FE**: Finally including both yields a **large negative** association, indicating that *within the same state over time*, holding annual trends constant, higher beer taxes correlate with significantly fewer traffic fatalities.  
- **R² vs Within R²**: A model’s overall R² can be **very large** if you soak up cross-sectional differences with fixed effects. But the **Within R²**—how much of the *year-to-year* variation within each state is explained—tends to be modest, highlighting that the main driver of fatality rates may be stable differences across states (captured by the FEs) rather than fluctuations in beer tax alone.

In [1]:
import pandas as pd
import pyfixest as pf

# 1. Load the Fatalities dataset
df = pd.read_csv("data/fatalities.csv")

# Optional: create a fatality rate per 100,000 population
df["fatal_rate"] = df["fatal"] / df["pop"] * 100000

# 2. Simple OLS (no fixed effects)
#    We'll regress fatal_rate on beer tax ('beertax')
fit_simple = pf.feols("fatal_rate ~ beertax", data=df)
print(fit_simple.summary())

# 3. OLS with state-level fixed effects
#    Note: 'state' is the panel index
fit_fe = pf.feols("fatal_rate ~ beertax | state", data=df)
print(fit_fe.summary())

# 4. Manually adding a dummy variable for each state
fit_dummy = pf.feols("fatal_rate ~ beertax + C(state)", data=df)
print(fit_dummy.summary())

# 5. OLS with time fixed effects
fit_fe_state_time = pf.feols("fatal_rate ~ beertax | year", data=df)
print(fit_fe_state_time.summary())

# 6. Manually adding dummies for each state & each year
fit_dummy_state_time = pf.feols("fatal_rate ~ beertax + C(year)", data=df)
print(fit_dummy_state_time.summary())

# 7. OLS with state and time fixed effects
fit_fe_state_time = pf.feols("fatal_rate ~ beertax | state + year", data=df)
print(fit_fe_state_time.summary())

# 8. Manually adding dummies for each state & each year
fit_dummy_state_time = pf.feols("fatal_rate ~ beertax + C(state) + C(year)", data=df)
print(fit_dummy_state_time.summary())


###

Estimation:  OLS
Dep. var.: fatal_rate, Fixed effects: 0
Inference:  iid
Observations:  336

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:--------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| Intercept     |     18.533 |        0.436 |    42.539 |      0.000 | 17.676 |  19.390 |
| beertax       |      3.646 |        0.622 |     5.865 |      0.000 |  2.423 |   4.869 |
---
RMSE: 5.421 R2: 0.093 
None
###

Estimation:  OLS
Dep. var.: fatal_rate, Fixed effects: state
Inference:  CRV1
Observations:  336

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |    2.5% |   97.5% |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|--------:|
| beertax       |     -6.559 |        2.914 |    -2.251 |      0.029 | -12.421 |  -0.696 |
---
RMSE: 1.755 R2: 0.905 R2 Within: 0.041 
None
###

Estimation:  OLS
Dep. var.: fatal_rate, Fixed effects: 0
Inference:  iid
Observa